<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Embedding_Chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from datasets import Dataset
import json
import numpy as np
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import torch




# dataset = load_dataset("b-mc2/sql-create-context", split="train")
# questions = dataset['question']
# print(questions)

# df = pd.read_json('data/GPT_Query_20240124.json')
# dataset = Dataset.from_pandas(df, preserve_index=False)
#
# client = chromadb.PersistentClient('DBAssyV3')
# collection = client.create_collection('VectorDB_AssyV3')
# model = SentenceTransformer('model/sentence-transformers')
#
# for i in range(len(dataset)):
#     collection.add(
#         embeddings=[model.encode(f"{dataset[i]['question']}").tolist()],
#         documents=[dataset[i]['context']],
#         metadatas=[{"answer":dataset[i]["answer"]}],
#         ids=[f"id_{i}"]
#     )

#Read data from DB

model = SentenceTransformer('model/sentence-transformers')
client = chromadb.PersistentClient('DBAssyV3')
collection = client.get_collection('VectorDB_AssyV3')

#Load DB & Read data from DB
# context = collection.query(
#   query_embeddings=[model.encode(user_question).tolist()],
#   n_results=10
# )



def getContext(question, collection, model):
    context = collection.query(
        query_embeddings=[model.encode(question).tolist()],
        n_results=1
    )
    text = context["documents"][0]
    distance = context["distances"][0]
    d =np.min(np.array(distance))
    answer = context['metadatas'][0]
    answer= answer[0]
    json_data = json.dumps(answer)
    return text[0], d, json_data

# model = SentenceTransformer('model/sentence-transformers')
# client = chromadb.PersistentClient('DB')
# collection = client.get_collection('QueryDB')
# user_question="What is the production today of the Assy V3 phase?"
# context = getContext(user_question,collection)
# print(context)

def getQuery(question, context, model, tokenizer):
    eval_prompt = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
    You must output the SQL query that answers the question.
    ### Input:{question}

    ### Context:{context}
    ### Response:
    """
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=1000)[0], skip_special_tokens=True))

    return eval_prompt

user_question = "What was the production yesterday of line 14 Assy V3?"

context, d, answer = getContext(user_question, collection, model)

#json_data = json.dumps(answer)
data = json.loads(answer)
print(data["answer"])
